I will now explore the city of Toronto using the preprocessed data I cleaned in the previous exercises. I will base this exploration on the New York City Neighborhoods exercise.

My Objective:
I will use the Foursquare API to explore neighborhoods in Toronto. I will use the *explore* function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the *k*-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

First, I will install the necessary packages and libraries:

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Here I am bringing in the preprocessing steps I did to get my desired dataframe:

In [8]:
file_path = 'https://raw.githubusercontent.com/d-alvear/Coursera_Capstone/master/CA_postalcodes_raw'
df_pc = pd.read_csv(file_path)

df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)

df_pc['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)

df_pc = df_pc.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']

df_pc.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
df_pc.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [9]:
file_path1 = 'https://raw.githubusercontent.com/d-alvear/Coursera_Capstone/master/Geospatial_Coordinates.csv'
df_ll = pd.read_csv(file_path1)

df_pc['Latitude'] = df_ll['Latitude']
df_pc['Longitude'] = df_ll['Longitude']
df_pc.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Now that the dataframe is created, I can start making a map of Toronto, with markers indicating neighborhoods:

In [18]:
latitude = 43.6532
longitude = -79.3832
# creating map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_pc['Latitude'], df_pc['Longitude'], df_pc['Borough'], df_pc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

To consolidate things and make the data more manageable, I will only be exploring neighborhoods that reside in the Downtown Toronto borough. So I will make a new dataframe consisting of these neighborhoods:

In [21]:
toronto_data = df_pc[df_pc['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


Now that I have the dataframe, I will generate a new map of the neighborhoods in Downtown Toronto:

In [31]:
latitude = 43.6532
longitude = -79.3832
# creating map of Downtown Toronto using latitude and longitude values
map_DTtoronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DTtoronto)  
    
map_DTtoronto

Now I will explore the first neighborhood in Downtown Toronto:

In [32]:
CLIENT_ID = 'IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q' # your Foursquare ID
CLIENT_SECRET = 'TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q
CLIENT_SECRET:TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN


In [33]:
toronto_data.loc[0, 'Neighborhood']

'Rosedale'

The first neighborhood is Rosedale and now I want the latitude and longitude so I can explore the venues in the area:

In [34]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


With the latitude and longitude, I will now search for the first 50 venues within 500 meters of the Rosedale neighborhood:

In [35]:
# type your answer here
LIMIT = 50
radius = 500

url = url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q&client_secret=TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=50'

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d153066bbf1bd00233a5939'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bae2150f964a520df873be3',
       'name': 'Mooredale House',
       'location': {'address': '146 Crescent Rd.',
        'crossStreet': 'btwn. Lamport Ave. and Mt. Pleasant Rd.',
        'lat': 43.678630645646535,
        'lng': -79.38009142511322,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678630645646535,
          'lng': -79.380091425113

I then use the 'get_category_type' for organizing the data:

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mooredale House,Building,43.678631,-79.380091
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In the above I was able to clean the resulting venue data and gain insight to the number and type of venues in the neighborhood.
Now I will do this for all the neighborhoods:

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

I create a new variable called 'toronto_venues' that runs the above function on every neighborhood in Downtown Toronto:

In [45]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [46]:
print(toronto_venues.shape)
toronto_venues.head()

(764, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


I then use the following to create a dataframe that tells me how many of each type of venue are present in each neighborhood:

In [56]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[0]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Adult Boutique,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


I group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [57]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.00,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.08,...,0.000000,0.00,0.00,0.0,0.00,0.02,0.000000,0.00,0.00,0.00
1,Berczy Park,0.00,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.000000,0.00,0.00,0.0,0.00,0.02,0.000000,0.00,0.00,0.00
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.00,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,...,0.000000,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00
3,"Cabbagetown, St. James Town",0.00,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.000000,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00
4,Central Bay Street,0.00,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.02,...,0.000000,0.00,0.00,0.0,0.00,0.02,0.000000,0.00,0.02,0.00
5,"Chinatown, Grange Park, Kensington Market",0.00,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.000000,0.00,0.02,0.0,0.00,0.06,0.000000,0.04,0.02,0.00
6,Christie,0.00,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.000000,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00
7,Church and Wellesley,0.02,0.02,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.000000,0.02,0.00,0.0,0.00,0.00,0.000000,0.02,0.00,0.02
8,"Commerce Court, Victoria Hotel",0.00,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.04,...,0.000000,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00
9,"Design Exchange, Toronto Dominion Centre",0.00,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.02,...,0.000000,0.00,0.00,0.0,0.02,0.00,0.000000,0.00,0.00,0.00


I then print each neighborhood along with the top 5 most common venues within them:

In [58]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0  American Restaurant  0.08
1                 Café  0.06
2           Steakhouse  0.06
3          Coffee Shop  0.06
4                Hotel  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.06
2          Café  0.04
3    Steakhouse  0.04
4      Beer Bar  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4             Plane  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.07
2                 Pub  0.04
3         Pizza Place  0.04
4  Italian Restaurant  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.06
2  Chinese Restaurant  0.04
3        Burger Joi

I then use the following function to sort the venues in descending order:

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Then I create a new dataframe that reports the top 10 venues of each neighborhood:

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",American Restaurant,Café,Steakhouse,Coffee Shop,Bar,Hotel,Breakfast Spot,Pizza Place,Asian Restaurant,Gastropub
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Cheese Shop,Café,Steakhouse,Farmers Market,Greek Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boat or Ferry,Boutique,Bar,Plane,Sculpture Garden
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Bakery,Park,Pizza Place,Pub,Italian Restaurant,Café,Gift Shop,Gastropub
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Spa,Chinese Restaurant,Burger Joint,Bubble Tea Shop,Dessert Shop,Discount Store,Seafood Restaurant


Finally I will cluster the neighborhoods. I will run k-means to create 5 clusters

In [63]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 0, 1, 4, 1, 2, 1, 4, 4], dtype=int32)

Here I create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,3,3,Park,Building,Playground,Trail,Concert Hall,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,1,Coffee Shop,Restaurant,Bakery,Park,Pizza Place,Pub,Italian Restaurant,Café,Gift Shop,Gastropub
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,1,Coffee Shop,Gastropub,Burger Joint,Men's Store,Japanese Restaurant,Gym,Gay Bar,Restaurant,Burrito Place,Ice Cream Shop
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,1,Coffee Shop,Pub,Park,Bakery,Restaurant,Theater,Breakfast Spot,Café,Mexican Restaurant,Cosmetics Shop
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,1,Café,Clothing Store,Coffee Shop,Ramen Restaurant,Italian Restaurant,Theater,Burrito Place,Shopping Mall,Burger Joint,Diner


I will now generate a map that visualizes this data:

In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters